In [1]:
import pandas as pd
import numpy as np

import random
from itertools import product

from sklearn.linear_model import LinearRegression
from catboost import CatBoostRegressor

from sklearn.metrics import mean_squared_error

from tqdm import tqdm
from joblib import Parallel, delayed
from tqdm_joblib import tqdm_joblib


/opt/anaconda3/lib/python3.12/site-packages/tqdm_joblib/__init__.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


### download data

In [3]:
# Download Data Dictionary
data_dict_path = '../../data/data_dictionary.xlsx'
data_dictionary = pd.read_excel(data_dict_path)

# Download Scoring File -- what the input data will be for the end
scoring_path = '../../data/scoring.xlsx'
scoring = pd.read_excel(scoring_path)

# Download Submission Format -- what the format of the output should be from my model
submission_format_path = '../../data/submission_format.csv'
submission_format = pd.read_csv(submission_format_path)

# Download Training Data -- what the training data is for this
training_path = '../../data/training.xlsx'
training = pd.read_excel(training_path).dropna()

### model functions

In [10]:
def fit_linear_regression(X, y):
    model = LinearRegression()
    model.fit(X, y)

    return model

class ConstantModel:
    # A fallback model that always predicts a constant value
    def __init__(self, value):
        self.value = value  # Store the constant target value

    def predict(self, X):
        # mimic predict
        return [self.value] * len

def fit_catboost(X, y):
    try:
        model = CatBoostRegressor(
            iterations=100,
            learning_rate=0.05,
            depth=6,
            cat_features=list(X.columns),
            verbose=0
        )

        model.fit(X, y)

        return model
    except:
        singular_value = y.iloc[0] if hasattr(y, 'iloc') else y[0]  # Extract the constant target value

        return lambda X_new: [singular_value] * len(X_new)  # Function that returns this value for any input


### download train and test data

In [11]:
car_data = training.copy()
car_data['Model Year'] = car_data['Model Year'].astype(str)

categorical_cols = car_data.select_dtypes(include=['object', 'category']).columns.tolist()
encoded_car_data =  pd.get_dummies(car_data, columns=categorical_cols)

train_indices = car_data.sample(frac=0.8, random_state=0).index
test_indices = car_data.drop(train_indices).index

train = car_data.loc[train_indices].copy()
test = car_data.loc[test_indices].copy()

train_encoded = encoded_car_data.loc[train_indices].copy()
test_encoded = encoded_car_data.loc[test_indices].copy()

### model testing functions

In [12]:
def get_models_by_basis(train_data, train_encoded, basis_columns, prediction_columns, target_col, model_func, encode):

    # Define the columns used to create different models
    basis_values = [train_data[col].unique().tolist() for col in basis_columns] # these are the potential values for each basis

    # Generate all possible (Model Year, Fuel Type) combinations as tuples
    basis_combinations = [tuple(values) for values in product(*basis_values)] # these are the combos of basis's so like 2020 Electric for example

    # Dictionary to store trained models, using (Model Year, Fuel Type) as the key
    trained_models = {}
    for basis_key in basis_combinations:

        basis_dict = {col: value for col, value in zip(basis_columns, basis_key)}
        # Filter training data for the specific Model Year & Fuel Type
        basis_indices = train_data[
            train_data[basis_dict.keys()].eq(pd.Series(basis_dict)).all(axis=1)
        ].index 

        # check if encoded
        model_data = train_encoded.copy() if encode else train_data.copy()

        # initialize X and y
        X = model_data.loc[basis_indices, prediction_columns]
        y = model_data.loc[basis_indices][target_col]


        if len(X) > 10: # if there are at least 25 values
            model = model_func(X, y)
            safe = True
        else:
            model = model_func(model_data[prediction_columns], model_data[target_col])
            safe = False

        # Store trained model using the (Model Year, Fuel Type) tuple as the key
        trained_models[basis_key] = (model, safe)

    return trained_models

In [13]:
def calculate_rmse(car_data, encoded_car_data, trained_models, basis_columns, prediction_columns, target_col, encode):
    all_prediction_rows = []
    for test_idx in test_indices:

        row = car_data.loc[test_idx]

        # get the basis values in the row (like 2020 for Model Year if Model Year is a basis)
        model_basis_dict = {col: row[col] for col in basis_columns}

        # get the model based on the basis
        model, safe = trained_models[tuple(model_basis_dict.values())]

        # get the encoded row
        if encode:
            formatted_row = encoded_car_data.loc[test_idx]
            formatted_row = formatted_row[prediction_columns]
        else:
            formatted_row = row[prediction_columns]

        # get the prediction vs. actual
        prediction = model.predict(pd.DataFrame(formatted_row).T)[0]
        actual = test.loc[test_idx, target_col]

        prediction_row = pd.DataFrame({
            'index': [test_idx],
            'prediction': [round(prediction, 0)],
            'actual': [actual],
            'safe': [safe]
        })
        all_prediction_rows.append(prediction_row)

    prediction_df = pd.concat(all_prediction_rows)
    rmse = np.sqrt(mean_squared_error(prediction_df['actual'], prediction_df['prediction']))

    return rmse

### model testing

Get the combinations of basis columns and prediction columns

In [14]:
column_options = list(set(train.columns) - {'Date', 'Vehicle Population'})

basis_prediction_combos = []
for _ in range(1000):
    # Select 1-3 random basis columns
    basis_columns = random.sample(column_options, random.randint(1, 3))
    
    # Select 3-7 prediction columns that are **not in basis_columns**
    remaining_columns = list(set(column_options) - set(basis_columns))
    prediction_columns = random.sample(remaining_columns, random.randint(3, 8-len(basis_columns)))
    
    # Store as a tuple
    basis_prediction_combos.append((basis_columns, prediction_columns))


Run the tests

In [15]:
# Define function to process each set of columns
model_func = fit_catboost
encode = False

def process_basis_prediction_combo(basis_columns, prediction_columns):
    target_col = 'Vehicle Population'
    
    # Ensure prediction columns do not include basis columns
    if encode:
        clean_prediction_columns = list(set(col for col in train_encoded.columns if col.split("_")[0] not in basis_columns) - {target_col, 'Date'})
    else:
        clean_prediction_columns = prediction_columns
    
    # Train models and calculate RMSE
    trained_models = get_models_by_basis(train, train_encoded, basis_columns, clean_prediction_columns, target_col, model_func, encode)
    rmse = calculate_rmse(car_data, encoded_car_data, trained_models, basis_columns, clean_prediction_columns, target_col, encode)

    # Store results as a DataFrame
    return pd.DataFrame({
        'basis_columns': [basis_columns],
        'prediction_columns': [prediction_columns],
        'rmse': [rmse]
    })

# Run in parallel using tqdm_joblib
with tqdm_joblib(tqdm(desc="Processing Models", total=len(basis_prediction_combos))):
   model_results_list = Parallel(n_jobs=-1)(delayed(process_basis_prediction_combo)(basis, pred) for basis, pred in basis_prediction_combos)


# Concatenate all results
model_results = pd.concat(model_results_list)
model_results.to_excel(fr'../../results/catboost_optim.xlsx', index=False)

  0%|          | 0/1000 [00:00<?, ?it/s]

AttributeError: 'function' object has no attribute 'predict'

In [10]:
model_results

,basis_columns,prediction_columns,rmse
0,"[Electric Mile Range, Number of Vehicles Regis...","[GVWR Class, Model Year, Vehicle Category]",17201.755380
0,"[Electric Mile Range, Number of Vehicles Regis...","[GVWR Class, Region, Vehicle Category, Fuel Te...",18046.970924
0,"[Region, Model Year, Number of Vehicles Regist...","[Vehicle Category, Fuel Type, GVWR Class]",16213.763188
0,[Vehicle Category],"[Region, Fuel Type, Number of Vehicles Registe...",8044.171736
0,"[Vehicle Category, Electric Mile Range]","[Model Year, Fuel Technology, Number of Vehicl...",16872.867224
0,[Region],"[Electric Mile Range, Model Year, Fuel Type, G...",8466.257413
0,[GVWR Class],"[Model Year, Fuel Type, Vehicle Category, Elec...",8317.726464
0,[Fuel Technology],[Number of Vehicles Registered at the Same Add...,8997.101708
0,[Electric Mile Range],"[Region, Model Year, Number of Vehicles Regist...",18543.359287
0,"[Vehicle Category, Region, Electric Mile Range]","[Fuel Technology, Fuel Type, GVWR Class, Numbe...",9394.415965


In [20]:
top_model_results = model_results.sort_values('rmse').head(10)
top_model_results

,basis_columns,prediction_columns,rmse
0,"[Vehicle Category, Fuel Type, Model Year]","[GVWR Class, Region, Electric Mile Range]",8296.070312
0,"[Model Year, Fuel Type, Vehicle Category]","[Fuel Technology, Electric Mile Range, Region]",8303.348196
0,"[Vehicle Category, Model Year, Fuel Type]","[Electric Mile Range, Number of Vehicles Regis...",8303.348196
0,"[GVWR Class, Fuel Type, Vehicle Category]","[Fuel Technology, Region, Number of Vehicles R...",9497.961403
0,"[Vehicle Category, Fuel Type, GVWR Class]","[Fuel Technology, Model Year, Region, Electric...",9498.020657
0,"[Fuel Type, Vehicle Category, Electric Mile Ra...","[Model Year, Region, Fuel Technology, GVWR Cla...",9646.000984
0,"[Electric Mile Range, Vehicle Category, Fuel T...","[Fuel Technology, Model Year, GVWR Class, Regi...",9646.000984
0,"[Vehicle Category, Fuel Technology, Fuel Type]",[Number of Vehicles Registered at the Same Add...,9646.093075
0,"[Vehicle Category, Fuel Type, Fuel Technology]","[GVWR Class, Region, Model Year]",9646.286129
0,"[Fuel Type, Fuel Technology, Vehicle Category]","[Region, GVWR Class, Electric Mile Range]",9646.286129


,Date,Model Year,Vehicle Population,Vehicle Category_B,Vehicle Category_BS,Vehicle Category_BT,Vehicle Category_MC,Vehicle Category_MH,Vehicle Category_P,Vehicle Category_T1,...,Electric Mile Range_51 to 100 miles,Electric Mile Range_>150 miles,Electric Mile Range_Not Applicable,Electric Mile Range_Unknown,Number of Vehicles Registered at the Same Address_1,Number of Vehicles Registered at the Same Address_2,Number of Vehicles Registered at the Same Address_3,Number of Vehicles Registered at the Same Address_Unknown,Number of Vehicles Registered at the Same Address_≥4,Region_Statewide
38210,2022,2006.0,1,False,False,False,False,False,False,False,...,False,False,True,False,False,True,False,False,False,True
23530,2021,2018.0,21,True,False,False,False,False,False,False,...,False,False,True,False,True,False,False,False,False,True
3301,2021,1993.0,4690,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,True,True
393,2021,2014.0,91108,False,False,False,False,False,True,False,...,False,False,True,False,False,False,True,False,False,True
5548,2020,2013.0,1694,False,False,False,False,False,False,False,...,False,False,True,False,False,False,True,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28568,2023,1982.0,7,False,False,False,False,False,False,False,...,False,False,True,False,False,True,False,False,False,True
17144,2019,2020.0,81,False,False,False,False,False,False,False,...,False,False,True,False,False,True,False,False,False,True
9586,2021,2010.0,455,False,False,False,False,False,False,False,...,False,False,True,False,False,True,False,False,False,True
28231,2021,1988.0,8,False,False,False,False,False,False,False,...,False,False,True,False,False,True,False,False,False,True
